# Load and Prepare Data
Load the CSV file using pandas, prepare features (X) and target (y), and split the data into training and testing sets.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV file
data = pd.read_csv('./Outputs/pop_sea_temp_fleet_amazon_oil.csv')

# Prepare features (X) and target (y)
X = data.drop(columns=['GMSL'])
y = data['GMSL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Model
Create and train a RandomForestRegressor, evaluate its performance using metrics like R2 score and MSE.

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Create and train the RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Print the evaluation metrics
print(f"Random Forest Model - Mean Squared Error: {mse_rf}")
print(f"Random Forest Model - R2 Score: {r2_rf}")

Random Forest Model - Mean Squared Error: 58.571252340576244
Random Forest Model - R2 Score: 0.9859929820979106


# Linear Regression Model
Implement linear regression using sklearn, evaluate model performance and analyze coefficients.

In [5]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Create and train the LinearRegression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

# Print the evaluation metrics
print(f"Linear Regression Model - Mean Squared Error: {mse_lr}")
print(f"Linear Regression Model - R2 Score: {r2_lr}")

# Analyze coefficients
X_train_sm = sm.add_constant(X_train)  # Add constant term for intercept
lr_model_sm = sm.OLS(y_train, X_train_sm).fit()

# Print the summary of the linear regression model
print(lr_model_sm.summary())

Linear Regression Model - Mean Squared Error: 75.92305171116169
Linear Regression Model - R2 Score: 0.9818433872932102
                            OLS Regression Results                            
Dep. Variable:                   GMSL   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     1071.
Date:                Mon, 18 Nov 2024   Prob (F-statistic):           3.27e-96
Time:                        23:39:30   Log-Likelihood:                -384.97
No. Observations:                 113   AIC:                             787.9
Df Residuals:                     104   BIC:                             812.5
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025     

# Logistic Regression Model
Transform the problem for logistic regression, train the model, and evaluate its performance.

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Transform the target variable for logistic regression
# Assuming GMSL is a continuous variable, we need to binarize it for logistic regression
# Here, we will use the median value to create two classes
median_gmsl = y.median()
y_binary = (y > median_gmsl).astype(int)

# Split the binary target variable into training and testing sets
y_train_binary = (y_train > median_gmsl).astype(int)
y_test_binary = (y_test > median_gmsl).astype(int)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the LogisticRegression model
log_reg_model = LogisticRegression(random_state=42)
log_reg_model.fit(X_train_scaled, y_train_binary)

# Make predictions
y_pred_log_reg = log_reg_model.predict(X_test_scaled)

# Evaluate the model
accuracy_log_reg = accuracy_score(y_test_binary, y_pred_log_reg)
conf_matrix_log_reg = confusion_matrix(y_test_binary, y_pred_log_reg)
class_report_log_reg = classification_report(y_test_binary, y_pred_log_reg)

# Print the evaluation metrics
print(f"Logistic Regression Model - Accuracy: {accuracy_log_reg}")
print("Confusion Matrix:")
print(conf_matrix_log_reg)
print("Classification Report:")
print(class_report_log_reg)

Logistic Regression Model - Accuracy: 1.0
Confusion Matrix:
[[16  0]
 [ 0 13]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        13

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



# Logistic Regression Model Classifier
Transform the problem for logistic regression, train the model, and evaluate its performance at classifying the GSML prediction as above or below the linear regression line.


In [17]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Create trendline using years as X and GMSL as y
years = data['year'].values.reshape(-1, 1)
gmsl_values = data['GMSL'].values

# Fit linear trend
trend_model = LinearRegression()
trend_model.fit(years, gmsl_values)

# Calculate expected GMSL values based on trendline
expected_gmsl = trend_model.predict(years)

# Create binary target: 1 if actual GMSL is above trend, 0 if below
y_binary = (gmsl_values > expected_gmsl).astype(int)

# Split maintaining same indices as your original split
y_train_binary = (y_train > trend_model.predict(data.loc[y_train.index, 'year'].values.reshape(-1,1))).astype(int)
y_test_binary = (y_test > trend_model.predict(data.loc[y_test.index, 'year'].values.reshape(-1,1))).astype(int)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression
log_reg_model = LogisticRegression(random_state=42)
log_reg_model.fit(X_train_scaled, y_train_binary)

# Make predictions
y_pred_log_reg = log_reg_model.predict(X_test_scaled)

# Evaluate model
accuracy_log_reg = accuracy_score(y_test_binary, y_pred_log_reg)
conf_matrix_log_reg = confusion_matrix(y_test_binary, y_pred_log_reg)
class_report_log_reg = classification_report(y_test_binary, y_pred_log_reg)

# Print evaluation metrics
print(f"Logistic Regression Model - Accuracy: {accuracy_log_reg}")
print("\nConfusion Matrix:")
print(conf_matrix_log_reg)
print("\nClassification Report:")
print(class_report_log_reg)

# For future predictions
future_years = np.array(range(2022, 2031)).reshape(-1, 1)
future_trend = trend_model.predict(future_years)
future_X_scaled = scaler.transform(future_X)
future_predictions_log = log_reg_model.predict(future_X_scaled)

print("\nFuture Predictions (1 = above trend, 0 = below trend):")
for year, pred in zip(future_years.flatten(), future_predictions_log):
    print(f"year {year}: {'Above' if pred == 1 else 'Below'} trend")

Logistic Regression Model - Accuracy: 0.8275862068965517

Confusion Matrix:
[[15  2]
 [ 3  9]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86        17
           1       0.82      0.75      0.78        12

    accuracy                           0.83        29
   macro avg       0.83      0.82      0.82        29
weighted avg       0.83      0.83      0.83        29


Future Predictions (1 = above trend, 0 = below trend):
year 2022: Above trend
year 2023: Above trend
year 2024: Above trend
year 2025: Above trend
year 2026: Above trend
year 2027: Above trend
year 2028: Above trend
year 2029: Above trend
year 2030: Above trend


# Future Predictions
Generate trend-based feature values for 2022-2030 and use all models to predict future GMSL values.

In [20]:
# Add these imports at the top
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Add this code after the existing linear regression but before future predictions

# Create trendline for classification boundary
years = data['year'].values.reshape(-1, 1)
gmsl_values = data['GMSL'].values
trend_model = LinearRegression()
trend_model.fit(years, gmsl_values)
trend_line = trend_model.predict(years)

# Create binary target (1 if above trend, 0 if below)
y_binary = (gmsl_values > trend_line).astype(int)

# Split binary target
y_train_binary = (y_train > trend_model.predict(data.loc[y_train.index, 'year'].values.reshape(-1,1))).astype(int)
y_test_binary = (y_test > trend_model.predict(data.loc[y_test.index, 'year'].values.reshape(-1,1))).astype(int)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train_scaled, y_train_binary)

# Predict future trends
future_years_array = future_years['year'].values.reshape(-1, 1)
future_trend = trend_model.predict(future_years_array)
future_X_scaled = scaler.transform(future_X)
future_predictions_log = log_model.predict(future_X_scaled)

# Add to results
future_years['Above_Trend'] = future_predictions_log

# Add these evaluation metrics before final print
print("\nLogistic Regression Model Evaluation:")
y_pred_log = log_model.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test_binary, y_pred_log):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_binary, y_pred_log))
print("\nClassification Report:")
print(classification_report(y_test_binary, y_pred_log))

# Add after existing code but before final prints
import os
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Create metrics for each model
rf_pred = rf_model.predict(X_test)
lr_pred = lr_model.predict(X_test)
log_pred_proba = log_model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
metrics = pd.DataFrame({
    'Random Forest': {
        'R2 Score': r2_score(y_test, rf_pred),
        'MSE': mean_squared_error(y_test, rf_pred),
        'MAE': mean_absolute_error(y_test, rf_pred),
        'Feature Importance': dict(zip(X.columns, rf_model.feature_importances_))
    },
    'Linear Regression': {
        'R2 Score': r2_score(y_test, lr_pred),
        'MSE': mean_squared_error(y_test, lr_pred),
        'MAE': mean_absolute_error(y_test, lr_pred),
        'Coefficients': dict(zip(X.columns, lr_model.coef_))
    },
    'Logistic Regression': {
        'Accuracy': accuracy_score(y_test_binary, y_pred_log),
        'Feature Coefficients': dict(zip(X.columns, log_model.coef_[0])),
        'Classification Report': classification_report(y_test_binary, y_pred_log)
    }
})

# Create output directory if it doesn't exist
os.makedirs('./Model_Outputs', exist_ok=True)

# Write comparison to file
with open('./Model_Outputs/model_comparison.txt', 'w') as f:
    f.write("Model Comparison Results\n")
    f.write("=======================\n\n")
    
    for model in metrics.columns:
        f.write(f"{model}\n")
        f.write("-" * len(model) + "\n")
        for metric, value in metrics[model].items():
            f.write(f"{metric}:\n")
            if isinstance(value, dict):
                for k, v in value.items():
                    f.write(f"  {k}: {v:.4f}\n")
            elif isinstance(value, str):
                f.write(f"{value}\n")
            else:
                f.write(f"{value:.4f}\n")
        f.write("\n")

print("\nModel comparison has been saved to ./Model_Outputs/model_comparison.txt")

# Modify final print to include logistic results
print("\nPredictions for 2022-2030:")
print(future_years)





# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# import statsmodels.api as sm

# # Load data
# data = pd.read_csv('./Outputs/pop_sea_temp_fleet_amazon_oil.csv')

# # Separate features and target
# X = data.drop(['GMSL', 'year'], axis=1)
# y = data['GMSL']

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train models
# rf_model = RandomForestRegressor(random_state=42)
# rf_model.fit(X_train, y_train)

# lr_model = LinearRegression()
# lr_model.fit(X_train, y_train)

# # Create future years dataframe
# future_years = pd.DataFrame()
# future_years['year'] = range(2022, 2031)

# # For each feature, extrapolate based on trend
# for column in X.columns:
#     yearly_change = data[column].diff().mean()
#     last_value = data[column].iloc[-1]
#     future_values = [last_value + (i * yearly_change) for i in range(1, 10)]
#     future_years[column] = future_values

# # Make predictions
# future_X = future_years[X.columns]
# future_predictions_rf = rf_model.predict(future_X)
# future_predictions_lr = lr_model.predict(future_X)

# # Add predictions to results
# future_years['GMSL_Prediction_RF'] = future_predictions_rf
# future_years['GMSL_Prediction_LR'] = future_predictions_lr

# # Print model statistics
# print("\nRandom Forest Feature Importance:")
# for feature, importance in zip(X.columns, rf_model.feature_importances_):
#     print(f"{feature}: {importance:.4f}")

# # For Linear Regression statistics using statsmodels
# X_with_constant = sm.add_constant(X_train)
# model = sm.OLS(y_train, X_with_constant).fit()
# print("\nLinear Regression Statistics:")
# print(model.summary())

# # Display predictions
# print("\nPredictions for 2022-2030:")
# print(future_years)


Logistic Regression Model Evaluation:
Accuracy: 0.8276

Confusion Matrix:
[[15  2]
 [ 3  9]]

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86        17
           1       0.82      0.75      0.78        12

    accuracy                           0.83        29
   macro avg       0.83      0.82      0.82        29
weighted avg       0.83      0.83      0.83        29


Model comparison has been saved to ./Model_Outputs/model_comparison.txt

Predictions for 2022-2030:
   year  Unnamed: 0           pop  popChangePercent    avgtemp  \
0  2022       142.0  7.955341e+09          0.876170  14.757305   
1  2023       143.0  8.001387e+09          0.882340  14.764610   
2  2024       144.0  8.047433e+09          0.888511  14.771915   
3  2025       145.0  8.093479e+09          0.894681  14.779220   
4  2026       146.0  8.139525e+09          0.900851  14.786525   
5  2027       147.0  8.185571e+09          0.907021  14.79